In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import pandas as pd
import re
from sklearn.model_selection import train_test_split

load_dotenv()

True

In [2]:
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

### 1. Instructions with Key Facts and Summary

In [5]:
# Function to format the instruction from key facts and event summary
def generate_instruction_from_facts_and_summary(key_facts, event_summary):
    # Format the key facts into a bullet point list
    formatted_facts = '\n'.join(key_facts)

    # Construct the instruction template with the provided facts and event summary
    instruction = f"""
Aşağıdaki gerçekler verildiğinde:
{formatted_facts}

Ve tarif edilen olay:
{event_summary}

Bu olayın daha geniş etkileri hakkında bir köşe yazısı yazın. Olayın kamu politikalarını, toplumsal tutumları veya gelecekteki gelişmeleri nasıl etkileyebileceğini göz önünde bulundurun.
    """
    return instruction.strip()

# Main function to extract key facts and event summary, then format the instruction
def generate_instruction(article_content):
    # Prepare the conversation state for GPT-4o chat-based completion
    conversation_state = [
        {"role": "system", "content": "Sen, aşağıdaki içeriğin yalnızca gerçeklere dayanan önemli noktalarını ve ana olayın gerçeklere dayanan bir özetini çıkarmakla görevlendirilmiş bir yardımcı asistansın. Orijinal makaleye herhangi bir şekilde atıfta bulunma. 'Makalede belirtiliyor ki' ya da 'Metne göre' gibi ifadelerden kaçın. Gerçek noktaları ve ana olayı tarafsız ve doğrudan bir şekilde sun. Yanıtı şu şekilde biçimlendir:\n\n**Önemli Gerçekler**:\n- Gerçek 1\n- Gerçek 2\n- Gerçek 3\n\n**Ana Olay Özeti**:\n- [Ana olayın kısa özeti]."},
        {"role": "user", "content": f"İçerik burada:\n\n{article_content}\n\nLütfen aşağıdakileri çıkar:\n1. Önemli gerçekler (makalede açıkça belirtilen olgusal noktalar).\n2. Ana olayın gerçeklere dayanan özeti (ana olayı nesnel ve tarafsız bir şekilde tanımla, yorum ya da öznel analiz ekleme)."}
    ]

    # Call GPT-4o chat-based completion API
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=conversation_state,
            max_tokens=600
        )

        # Extract the response from the assistant
        output = response.choices[0].message.content.strip()

        # Use case-insensitive flag via re.IGNORECASE, and clean up the expression
        key_facts_match = re.search(r"\*\*Önemli Gerçekler\*\*([\s\S]*?)\*\*Ana Olay Özeti\*\*", output, re.IGNORECASE)
        event_summary_match = re.search(r"\*\*Ana Olay Özeti\*\*([\s\S]*)", output, re.IGNORECASE)
        
        if key_facts_match:
            # Extract key facts (assuming each fact starts with a hyphen "-")
            key_facts_section = key_facts_match.group(1)
            key_facts = [fact.strip() for fact in key_facts_section.split("\n") if fact.strip().startswith('-')]
        else:
            key_facts = []

        if event_summary_match:
            event_summary = event_summary_match.group(1).strip()

            # Remove any leading colon or unwanted characters before the summary
            event_summary = re.sub(r"^[:\-\s]+", "", event_summary).strip()
        else:
            event_summary = ""

        # Format the final instruction using the key facts and event summary
        instruction = generate_instruction_from_facts_and_summary(key_facts, event_summary)
        return instruction


    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage with one article's content
article_content = """
Alman medyasında Türkiye hakkında bir haber çıktığında hepimiz önce bir irkiliriz.
Acaba terör örgütlerinden hangisini öveceğini düşünürüz.
Bu Almanya'ya karşı bir önyargı değil, yılların getirdiği bir deneyimin sonucudur.
Alman medyası, PKK'dan DHKP/ C'ye, FETÖ'den MLKP'ye nerede Türkiye'nin başına bela olmuş bir terör örgütü varsa onu övmesi ve meşrulaştırmasıyla meşhurdur.
Ancak Alman medyasında Türkiye hakkında yer alan bir haber hepimizi biraz şaşırttı.
Haberi veren Spiegel dergisi, Almanya'nın Türkiye'ye torpido ve güdümlü füze ihracatı için izin verdiğini açıkladı.
Alman hükümetinde bir değişiklik olmadığı, paralel bir evrene de geçmediğimize göre bu oldukça önemli bir haberdi.
Çünkü Almanya merkezli başka bir haber de Siemens firmasının Akkuyu Nükleer Santrali'miz için gerekli ekipmanları yollamadığı gündemdeyken çok sayıda Türkiye düşmanı bakana sahip Alman hükümeti bu izni vermişti.
Cevap aslında oldukça basit ama bir o kadar da önemli: Çünkü artık biz üretiyoruz.
Türk savunma sanayii, Cumhurbaşkanı Erdoğan önderliğinde Türk mühendislerinin insanüstü çabasıyla yurtdışında paramızla bize verilmeyen hangi ürün varsa onu üretmek için var gücüyle çalışıp ambargolarını başlarına çaldı.
O yüzden artık istediğimiz ürünü vermediklerinde çok da dert etmiyoruz.
ABD ve İsrail'in bize vaktiyle İHA vermeyip Türkiye'yi dünyanın en büyük İHA üreticisi yapmalarında olduğu gibi onlarca örnek var.
Mesela Kanada, SİHA'larımız için Elektro-Optik Keşif, Gözetleme ve Hedef Tespit Sistemi CATS'ların satışını yasaklayarak bizi bu sektöre de sokmuştu.
Hatta artık Kanada'dan daha kaliteli ürünler geliştirmiştik.
Ve evet, Kanada da ""ambargosunu"" kaldırmıştı.
Bugün de Almanya'nın bize vaktiyle vermediği torpido ve güdümlü füzeleri üretmeye başladık.
Almanya o yüzden bu satışa izin vererek Türkiye'deki Alman hayranlarını şaşırttı.
Terör örgütü PKK ile mücadele ettiğimiz için parasıyla sattığı tankların parçalarını yollamayan, Türkiye'nin enerji bağımsızlığı için gerekli olan barajları engellemek için her türlü imkânını seferber eden Almanya'dan benzer haberler göreceğiz.
Bu zamana kadar vermek istemediği ne kadar savunma sanayii ürünü varsa şimdi satmak için kendisi çabalayacak.
Bu nokta bizi rehavete sevk etmemeli.
Elbette ki Türk Silahlı Kuvvetleri'mizin ihtiyaçlarının karşılanması gerekiyor ve bu yapılmalı.
Ancak satın aldığımız her ürünün daha iyisini kendi imkânlarımızla üretme amacımızdan bir an bile vazgeçmemeliyiz.
Biz bu hedeften sapmadığımız sürece düne kadar kırk dereden su getiren ""müttefiklerimizin"" kapımızı aşındırdığını göreceğiz.
Çünkü artık biz üretiyoruz.
"""

# Generate the instruction (key facts and event summary) and response
instruction = generate_instruction(article_content)
response = article_content.strip()
print(instruction)
print()
print(response)

Aşağıdaki gerçekler verildiğinde:
- Alman dergisi Spiegel, Almanya'nın Türkiye'ye torpido ve güdümlü füze ihracatı için izin verdiğini bildirdi.
- Siemens, Akkuyu Nükleer Santrali için gerekli ekipmanları göndermedi.
- Türkiye, savunma sanayii alanında dışa bağımlılığı azaltmak için önemli adımlar attı ve bazı ürünlerin üretimine başladı.
- Kanada, Türkiye'ye yönelik Elektro-Optik Keşif, Gözetleme ve Hedef Tespit Sistemi satışını yasaklamıştı, ancak Türkiye alternatiflerini geliştirmiş ve Kanada ambargosunu kaldırmıştı.
- Türkiye, Almanya'nın daha önce vermediği torpido ve güdümlü füzeleri üretmeye başlamıştır.

Ve tarif edilen olay:
Almanya, Türkiye'ye torpido ve güdümlü füze ihracatı için izin verdi. Bu gelişme, Türkiye'nin savunma sanayiinde kendi üretim kapasitesini artırması sonucunda, Almanya'nın daha önce uyguladığı kısıtlamaları yumuşatmasına işaret ediyor.

Bu olayın daha geniş etkileri hakkında bir köşe yazısı yazın. Olayın kamu politikalarını, toplumsal tutumları veya gelece

In [11]:
columnist_name = "hilalkaplan"

df = pd.read_csv(f"../../columnist_data/cleaned_articles/{columnist_name}_cleaned_articles.csv")

# Filter out rows where the article content is "Content not found"
df = df[df['Article Content'] != "Content not found"].reset_index(drop=True)

# Initialize lists to store the generated instructions and formatted responses
instructions = []
responses = []

# Iterate through each row in the filtered DataFrame
for row in df.itertuples():
    index = row[0]
    article_content = row[4].strip()

    # Generate the instruction for the article content
    instruction = generate_instruction(article_content)
    
    if instruction:
        # Format the original article content as a response
        formatted_response = article_content.strip()
        
        # Append the generated instruction and formatted response to their respective lists
        instructions.append(instruction)
        responses.append(formatted_response)

        # Optional: Print progress
        print(f"Processed article {index + 1}/{len(df)}")

# Add the generated instructions and formatted responses to the DataFrame
df['Instruction'] = instructions
df['Response'] = responses

# Select only the required columns
df_final = df[['Article Title', 'Date', 'URL', 'Instruction', 'Response']]

# Save the updated DataFrame to a new CSV file
df_final.to_csv(f'../../finetune_data/{columnist_name}/{columnist_name}.csv', index=False)
print("Filtered instructions and responses generated and saved to CSV successfully.")

Processed article 1/1464
Processed article 2/1464
Processed article 3/1464
Processed article 4/1464
Processed article 5/1464
Processed article 6/1464
Processed article 7/1464
Processed article 8/1464
Processed article 9/1464
Processed article 10/1464
Processed article 11/1464
Processed article 12/1464
Processed article 13/1464
Processed article 14/1464
Processed article 15/1464
Processed article 16/1464
Processed article 17/1464
Processed article 18/1464
Processed article 19/1464
Processed article 20/1464
Processed article 21/1464
Processed article 22/1464
Processed article 23/1464
Processed article 24/1464
Processed article 25/1464
Processed article 26/1464
Processed article 27/1464
Processed article 28/1464
Processed article 29/1464
Processed article 30/1464
Processed article 31/1464
Processed article 32/1464
Processed article 33/1464
Processed article 34/1464
Processed article 35/1464
Processed article 36/1464
Processed article 37/1464
Processed article 38/1464
Processed article 39/

In [18]:
# Split the dataset into training and test sets
columnist_name = "hilalkaplan"

# Load the full dataset with instructions and responses
df = pd.read_csv(f'../../finetune_data/{columnist_name}/{columnist_name}.csv')

# Split the data into train (80%) and test (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the training and test sets to separate CSV files
train_df.to_csv(f'../../finetune_data/{columnist_name}/{columnist_name}_train.csv', index=False)
test_df.to_csv(f'../../finetune_data/{columnist_name}/{columnist_name}_test.csv', index=False)

print("Training and test datasets saved successfully.")

Training and test datasets saved successfully.


### 2. Instructions with Shorter Summary

In [12]:
system_prompt = "Sen, sana verilecek olan içeriğin yalnızca gerçeklere dayanan önemli noktalarını ve ana olayın gerçeklere dayanan bir özetini çıkarmakla görevlendirilmiş bir yardımcı asistansın. Orijinal makaleye herhangi bir şekilde atıfta bulunma. 'Makalede belirtiliyor ki' ya da 'Metne göre' gibi ifadelerden kaçın. Yazarın öznel fikirlerini özete dahil etme. Gerçek noktaları ve ana olayı tarafsız ve doğrudan bir şekilde sun. Lütfen cevabında sadece özeti bulundur."
user_prompt_template = "İçerik burada:\n\n{article_content}\n\n Lütfen ana olayın özetini çıkarın (ana olayı nesnel ve tarafsız bir şekilde tanımla, yorum ya da öznel analiz ekleme). Cevabınızı 2 cümle ile sınırlayın."
instruction_template = "Lütfen aşağıdaki olay hakkında bir köşe yazısı yazın:\n\n{event_summary}"

In [15]:
# Main function to extract key facts and event summary, then format the instruction
def generate_instruction(article_content):
    # Prepare the conversation state for GPT-4o chat-based completion
    conversation_state = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_template.format(article_content=article_content)}
    ]

    # Call GPT-4o chat-based completion API
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=conversation_state,
            max_tokens=200
        )

        # Extract the response from the assistant
        output = response.choices[0].message.content.strip()

        # Format the final instruction using the key facts and event summary
        instruction = instruction_template.format(event_summary=output)
        return instruction


    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage with one article's content
article_content = """
Alman medyasında Türkiye hakkında bir haber çıktığında hepimiz önce bir irkiliriz.
Acaba terör örgütlerinden hangisini öveceğini düşünürüz.
Bu Almanya'ya karşı bir önyargı değil, yılların getirdiği bir deneyimin sonucudur.
Alman medyası, PKK'dan DHKP/ C'ye, FETÖ'den MLKP'ye nerede Türkiye'nin başına bela olmuş bir terör örgütü varsa onu övmesi ve meşrulaştırmasıyla meşhurdur.
Ancak Alman medyasında Türkiye hakkında yer alan bir haber hepimizi biraz şaşırttı.
Haberi veren Spiegel dergisi, Almanya'nın Türkiye'ye torpido ve güdümlü füze ihracatı için izin verdiğini açıkladı.
Alman hükümetinde bir değişiklik olmadığı, paralel bir evrene de geçmediğimize göre bu oldukça önemli bir haberdi.
Çünkü Almanya merkezli başka bir haber de Siemens firmasının Akkuyu Nükleer Santrali'miz için gerekli ekipmanları yollamadığı gündemdeyken çok sayıda Türkiye düşmanı bakana sahip Alman hükümeti bu izni vermişti.
Cevap aslında oldukça basit ama bir o kadar da önemli: Çünkü artık biz üretiyoruz.
Türk savunma sanayii, Cumhurbaşkanı Erdoğan önderliğinde Türk mühendislerinin insanüstü çabasıyla yurtdışında paramızla bize verilmeyen hangi ürün varsa onu üretmek için var gücüyle çalışıp ambargolarını başlarına çaldı.
O yüzden artık istediğimiz ürünü vermediklerinde çok da dert etmiyoruz.
ABD ve İsrail'in bize vaktiyle İHA vermeyip Türkiye'yi dünyanın en büyük İHA üreticisi yapmalarında olduğu gibi onlarca örnek var.
Mesela Kanada, SİHA'larımız için Elektro-Optik Keşif, Gözetleme ve Hedef Tespit Sistemi CATS'ların satışını yasaklayarak bizi bu sektöre de sokmuştu.
Hatta artık Kanada'dan daha kaliteli ürünler geliştirmiştik.
Ve evet, Kanada da ""ambargosunu"" kaldırmıştı.
Bugün de Almanya'nın bize vaktiyle vermediği torpido ve güdümlü füzeleri üretmeye başladık.
Almanya o yüzden bu satışa izin vererek Türkiye'deki Alman hayranlarını şaşırttı.
Terör örgütü PKK ile mücadele ettiğimiz için parasıyla sattığı tankların parçalarını yollamayan, Türkiye'nin enerji bağımsızlığı için gerekli olan barajları engellemek için her türlü imkânını seferber eden Almanya'dan benzer haberler göreceğiz.
Bu zamana kadar vermek istemediği ne kadar savunma sanayii ürünü varsa şimdi satmak için kendisi çabalayacak.
Bu nokta bizi rehavete sevk etmemeli.
Elbette ki Türk Silahlı Kuvvetleri'mizin ihtiyaçlarının karşılanması gerekiyor ve bu yapılmalı.
Ancak satın aldığımız her ürünün daha iyisini kendi imkânlarımızla üretme amacımızdan bir an bile vazgeçmemeliyiz.
Biz bu hedeften sapmadığımız sürece düne kadar kırk dereden su getiren ""müttefiklerimizin"" kapımızı aşındırdığını göreceğiz.
Çünkü artık biz üretiyoruz.
"""


article_content_2 = """İstanbul Büyükşehir Belediyesi'ne bağlı, internet üzerinden satış yapan İstanbul Kitap, Türkiye'de satılması yasak olan, eski PKK kurucularından Sakine Cansız'ın kitabını satıyormuş.
Skandal ortaya çıkınca, gelen tepkiler üzerine İmamoğlu ekibi panikle sayfayı kapatmış, satışına son vermişti.
Yine hafta başında, yargılanan HDP eski eş başkanı Demirtaş'ın yazdığı kitabın tiyatro oyununu Demirtaş'ın eşi, Kılıçdaroğlu ve İmamoğlu'nun eşleri birlikte izleyip kameralara poz vermişti.
Elbette HDP Eş Başkanı Buldan'ın ve CHP İl Başkanı Kaftancıoğlu'nun da orada olması siyasi mesaj kaygısının göstergesiydi.
Nitekim Kaftancıoğlu, yine hafta başında PKK'nın gazetesi Yeni Yaşam'a röportaj vermiş, gazete de röportajı manşetten ""Yan yana durmalıyız"" şeklinde duyurmuştu.
CHP ve İyi Parti'nin yerel seçimlerdeki zımni ortakları HDP ile araları bir açılıp bir düzeliyor.
""Ne seninle ne de sensiz"" diye sevgilileri hatırlatan bu durumu ne kadar içlerine sindirecekler bilmiyorum.
Ancak bu gidişatın 'kavgada söylenmez' denecek zirvelerinden birini hatırlatmak isterim.
HDP Grup Başkanvekili Fatma Kurtulan, meclis çatışı altında, genel kurulun kürsüsünden açıkça şunları söylemişti: ""İyi Parti, size söylüyorum: Size rağmen, içinde bulunduğunuz ittifaka, HDP ve PKK'ya içinde gönül vermişlerin de olduğu insanlar oy verdi.
Şu an koltuklarınızda HDP'nin oylarıyla oturuyorsunuz.
Bu ittifakta, CHP ile yaptığınız ittifakta HDP'nin oylarının etkisi vardır."" CHP ve İyi Parti'nin zımni ittifak ortağı olan HDPKK, ara sıra 'hizaya getirmek'için böyle tepkiler veriyor.
Mesele geçen seneki Cumhuriyet Bayramı'nda da CHP İsviçre Birliği'nin Zürih'te düzenlediği kutlamayı PKK sempatizanları basmıştı.
İçeriye giren PKK yandaşları, Öcalan pankartını açıp slogan attılar.
CHP İstanbul İl Başkanı Canan Kaftancıoğlu ve CHP Grup Başkanvekili Engin Özkoç'un da bulunduğu salondan PKK yandaşlarını kovmak yerine, CHP İsviçre Birliği Yönetim Kurulu Üyesi Ünal Konakçı, ""Açıklamanızı saygıyla karşıladık sizi dinledik.
Siz bizi dost olarak görmeyebilirsiniz, biz sizi dostumuz olarak görüyoruz"" şeklinde konuşmuştu.
Gezi kalkışması sırasında Atatürk ile Öcalan posterlerinin yan yana getirilmesiyle başlayan süreç, ""Her eve HDP'ye bir oy"" ile devam etti.
Ve bugün terörü meşrulaştırıp, PKK sempatizanlarına 'dostumuz' diyen bir vasıta gelindi.
Bakalım Erdoğan düşmanlığı yüzünden daha ne hallere düşülecek?
"""
# Generate the instruction (key facts and event summary) and response
instruction = generate_instruction(article_content_2)
response = article_content.strip()
print(instruction)

Lütfen aşağıdaki olay hakkında bir köşe yazısı yazın:

İstanbul Kitap, Sakine Cansız'ın kitabını satışa sununca tepkiler sonucu satışını durdurdu. Ayrıca, CHP ve İyi Parti'nin HDP ile olan ilişkileri ve geçtiğimiz günlerde yaşanan etkinliklerdeki duruşları, siyasi ortaklıklarına dair tartışmalara yol açtı.


In [17]:
columnist_name = "hilalkaplan"

df = pd.read_csv(f"../../columnist_data/cleaned_articles/{columnist_name}_cleaned_articles.csv")

# Filter out rows where the article content is "Content not found"
df = df[df['Article Content'] != "Content not found"].reset_index(drop=True)

# Initialize lists to store the generated instructions and formatted responses
summaries = []
instructions = []
responses = []

# Iterate through each row in the filtered DataFrame
for row in df.itertuples():
    index = row[0]
    article_content = row[4].strip()

    # Generate the instruction for the article content
    instruction = generate_instruction(article_content)

    if instruction:
        summary = instruction.split("\n\n")[-1].strip()
        # Format the original article content as a response
        formatted_response = article_content.strip()
        
        # Append the generated instruction and formatted response to their respective lists
        summaries.append(summary)
        instructions.append(instruction)
        responses.append(formatted_response)

        # Optional: Print progress
        print(f"Processed article {index + 1}/{len(df)}")

# Add the generated instructions and formatted responses to the DataFrame
df["Summary"] = summaries
df['Instruction'] = instructions
df['Response'] = responses

# Select only the required columns
df_final = df[['Article Title', 'Date', 'URL', 'Summary', 'Instruction', 'Response']]

# Save the updated DataFrame to a new CSV file
df_final.to_csv(f'../../finetune_data/{columnist_name}/{columnist_name}.csv', index=False)
print("Filtered instructions and responses generated and saved to CSV successfully.")

Processed article 1/1464
Processed article 2/1464
Processed article 3/1464
Processed article 4/1464
Processed article 5/1464
Processed article 6/1464
Processed article 7/1464
Processed article 8/1464
Processed article 9/1464
Processed article 10/1464
Processed article 11/1464
Processed article 12/1464
Processed article 13/1464
Processed article 14/1464
Processed article 15/1464
Processed article 16/1464
Processed article 17/1464
Processed article 18/1464
Processed article 19/1464
Processed article 20/1464
Processed article 21/1464
Processed article 22/1464
Processed article 23/1464
Processed article 24/1464
Processed article 25/1464
Processed article 26/1464
Processed article 27/1464
Processed article 28/1464
Processed article 29/1464
Processed article 30/1464
Processed article 31/1464
Processed article 32/1464
Processed article 33/1464
Processed article 34/1464
Processed article 35/1464
Processed article 36/1464
Processed article 37/1464
Processed article 38/1464
Processed article 39/

In [18]:
# Split the dataset into training and test sets
columnist_name = "hilalkaplan"

# Load the full dataset with instructions and responses
df = pd.read_csv(f'../../finetune_data/{columnist_name}/{columnist_name}.csv')

# Split the data into train (80%) and test (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the training and test sets to separate CSV files
train_df.to_csv(f'../../finetune_data/{columnist_name}/{columnist_name}_train.csv', index=False)
test_df.to_csv(f'../../finetune_data/{columnist_name}/{columnist_name}_test.csv', index=False)

print("Training and test datasets saved successfully.")

Training and test datasets saved successfully.
